In [2]:
import copy
import itertools

import numpy as np
# Imports
import pandas as pd
import re
import math
import collections

import tqdm

In [3]:
# Load DB
df_inter = pd.read_pickle("../data/dataframes/inter.pkl")

In [6]:
# Visu DB
df_inter[df_inter.AF.apply(isinstance, args=(float,))]

,PT,AU,AF,TI,SO,DT,DE,ID,AB,C1,...,Z9,PY,WC,DI,SC,UT,CN,lst_DT,lst_Target,lst_SDG


In [4]:
# Visu Authors-EM
test = df_inter[["AF", "EM"]]
test

,AF,EM
6,"Kao, Chih-Han; Wang, Chien-Chih; Wang, Her -Yung",ccw@gcloud.csu.edu.tw
28,"Nugra, Hugo; Abad, Alejandra; Fuertes, Walter;...",henugra@espe.edu.ec; maabad2@espe.edu.ec; wmfu...
90,"Peng, Yaoqi; Xiao, Yingxin; Fu, Zetian; Dong, ...",lxxcau@cau.edu.cn
101,"Alvarez-de-los-Mozos, Esther; Renteria-Bilbao,...",esther.alvarez@deusto.es; arantxa.renteria@tec...
218,"Laskurain-Iturbe, Iker; Arana-Landin, German; ...",iker.laskurain@ehu.eus; g.arana@ehu.eus; benat...
...,...,...
1244072,"Strengers, Yolande; Nicholls, Larissa",yolande.strengers@rmit.edu.au
1244214,"Kan, Man Yee; Sullivan, Oriel; Gershuny, Jonathan",oriel.sullivan@sociology.ox.ac.uk
1244296,"Cornwell, Benjamin; Gershuny, Jonathan; Sulliv...",cornwell@cornell.edu; j.gershuny@ucl.ac.uk; o....
1244855,"Abdulcadir, Jasmine; Dewaele, Romain; Firmenic...",jasmine.abdulcadir@hcuge.ch


In [9]:
import re
import itertools
import copy
def match_email_name(emails, names ) -> dict:
    dic_mails_names = dict()
    # If one of the two input lists are NAN (type==float), we return an empty dic
    if type(emails) == float or type(names) == float:
        return dic_mails_names

    # They need to be set, to avoid repetition
    # Sorting
    names.sort(key = len, reverse=True)
    names = set(names)
    emails = {e.lower() for e in set(emails)}

    def check_names_in_email(e:str, mode ="full_word"):
        """
        Intermediate function, embedded. For each email, we have 3 ways of looking for a match (3 modes)
        combi: first_name and last_name in email or firstname[0] + last_name or lastname[0]+ firstname
        full_word : the full_name is split into words, and one of these words must be in the email
        chunk : At least the first 3 characters of a word must be in the email
        ini: first letter of each word must be in the email (Kevin Michoud ==> km or mk)
        Args:
            e:
            mode: str 'full_word', 'chunk', 'ini', combi

        Returns: None

        """
        for fn in copy.deepcopy(names):
            rootless_email = e.split("@")[0]
            sn = [s for s in re.split('\W+', fn) if len(s)>0]

            if mode == "combi" and len(sn) == 2:
                # check if combo of firstname + lastname in rootless email
                if (sn[0].lower() in rootless_email and sn[1][0].lower() in rootless_email) or \
                        (sn[0][0].lower() in rootless_email and sn[1].lower() in rootless_email):
                    dic_mails_names[e] = fn
                    emails.remove(e)
                    names.remove(fn)
                    return None
            for w in [w for w in sn if len(w)>2]:
                # First
                if mode == "full_word":
                    if w.lower() in rootless_email:
                        # If the word in the name of the author
                        # is present in the email and at least 3 characters, we consider that they are linked

                        # update dict, remove from both sets, and skip the other words
                        # in the name
                        dic_mails_names[e] = fn
                        emails.remove(e)
                        names.remove(fn)
                        return None
                elif mode == "chunk":
                    if w[0:2].lower() in rootless_email:
                        # If at least 3 characters of a word in the name of the author
                        # is present in the email, we consider that they are linked

                        # update dict, remove from both sets, and skip the other words
                        # in the name
                        dic_mails_names[e] = fn
                        emails.remove(e)
                        names.remove(fn)
                        return None
            if mode == "ini":
            # Looking for a variation of first letter of firstnames and lastnames
                initials = [w[0].lower() for w in sn if len(w) >0]
                for combo in set(itertools.permutations(initials)):
                    if "".join(combo) in rootless_email:
                        dic_mails_names[e] = fn
                        emails.remove(e)
                        names.remove(fn)
                        return None
                # Case sn = ['Ye', 'Ben', 'Haobin'] ==> em = 'yehb3@mail.sysu.edu.cn'
                # Check if one of the word has len == 2, and consider initial as two letters
                doit = False
                for w in sn:
                    if len(w) == 2:
                        doit = True
                if doit:
                    initials = []
                    for w in sn:
                        if 0 < len(w) < 3:
                            initials.append(w.lower())
                        elif len(w) > 2:
                            initials.append(w[0].lower())
                    for combo in set(itertools.permutations(initials)):
                        if "".join(combo) in rootless_email:
                            dic_mails_names[e] = fn
                            emails.remove(e)
                            names.remove(fn)
                            return None


    # We go through once first, looking for combinations of lastnames and firstnames when ONLY two words in a name
    for email in copy.deepcopy(emails):
        check_names_in_email(e = email, mode="combi")
    # We go through once first, looking for full_word of more than 2 letters
    for email in copy.deepcopy(emails):
        check_names_in_email(e = email, mode="full_word")
    # Then, for chunk of words of at least 3 characters
    for email in copy.deepcopy(emails):
        check_names_in_email(e = email, mode="chunk")

    # Lastly, we check for initials
    for email in copy.deepcopy(emails):
        check_names_in_email(e = email, mode="ini")

    # Once, we've done the search, if there's only one email and one name left in the sets,
    # We match them
    if len(names) == len(emails) == 1:
        dic_mails_names[emails.pop()] = names.pop()

    # Tag emails left as "no_match"
    if len(emails) > 0:
        for email in emails:
            dic_mails_names[email] = "no_match"
    return dic_mails_names


In [10]:
lst_mail = ['m.bahri@impetial.ac.uk', 'e.o-sullivan16@imperial.ac.uk', 'shunwang.gong16@imperialac.uk', 'isliuf1990@gmail.com', 'liuxm@cse.msu.edu', 'm.bronstein@imperiaLac.uk', 's.zafeiriou@imperial.ac.uk']
lst_name = ['Bahri, Mehdi', "O' Sullivan, Eimear", 'Gong, Shunwang', 'Liu, Feng', 'Liu, Xiaoming', 'Bronstein, Michael M.', 'Zafeiriou, Stefanos']
match_email_name(lst_mail, lst_name)
fn = 'Kruk, A.'
sn = [s for s in re.split('\W+', fn) if len(s)>0]
sn

['Kruk', 'A']

In [11]:
test = df_inter[["AF", "EM"]]
test = test.sample(1)
af= test.loc[:, "AF"].values[0].split("; ")
em = test.loc[:, "EM"].values[0].split("; ")

print('Names', af)
print("Emails", em)
print(match_email_name(emails=em, names=af))


Names ['Sanchez-Sepulveda, Monica, V', 'Torres-Kompen, Ricardo', 'Fonseca, David', 'Franquesa-Sanchez, Jordi']
Emails ['monica.sanchez@salle.url.edu', 'ricardo.torres@salle.url.edu', 'david.fonseca@salle.url.edu', 'jordi.franquesa@upc.edu']
{'david.fonseca@salle.url.edu': 'Fonseca, David', 'monica.sanchez@salle.url.edu': 'Franquesa-Sanchez, Jordi', 'ricardo.torres@salle.url.edu': 'Torres-Kompen, Ricardo', 'jordi.franquesa@upc.edu': 'Sanchez-Sepulveda, Monica, V'}


In [12]:
%%time
df_inter = df_inter.assign(lst_AF=lambda x: x.AF.str.split("; "))
df_inter = df_inter.assign(lst_EM=lambda x: x.EM.str.split("; "))
df_inter['email']= np.vectorize(match_email_name)(df_inter['lst_EM'], df_inter['lst_AF'])


ValueError: not enough values to unpack (expected 2, got 0)

In [13]:
test = df_inter[['lst_AF', "email"]]
test

,lst_AF,email
6,"[Wang, Chien-Chih, Wang, Her -Yung, Kao, Chih-...","{'ccw@gcloud.csu.edu.tw': 'Wang, Chien-Chih'}"
28,"[Toulkeridis, Theofilos, Galarraga, Fernando, ...","{'henugra@espe.edu.ec': 'Nugra, Hugo', 'ttoulk..."
90,"[Zheng, Yongjun, Xiao, Yingxin, Dong, Yuhong, ...","{'lxxcau@cau.edu.cn': 'Li, Xinxing'}"
101,"[Alvarez-de-los-Mozos, Esther, Renteria-Bilbao...","{'fernando.diaz@deusto.es': 'Diaz-Martin, Fern..."
218,"[Uriarte-Gallastegi, Naiara, Laskurain-Iturbe,...","{'iker.laskurain@ehu.eus': 'Laskurain-Iturbe, ..."
...,...,...
1244072,"[Strengers, Yolande, Nicholls, Larissa]","{'yolande.strengers@rmit.edu.au': 'Strengers, ..."
1244214,"[Gershuny, Jonathan, Sullivan, Oriel, Kan, Man...",{'oriel.sullivan@sociology.ox.ac.uk': 'Sulliva...
1244296,"[Cornwell, Benjamin, Gershuny, Jonathan, Sulli...","{'o.sullivan@ucl.ac.uk': 'Sullivan, Oriel', 'j..."
1244855,"[Abdulcadir, Jasmine, Firmenich, Natacha, Bots...","{'jasmine.abdulcadir@hcuge.ch': 'Abdulcadir, J..."


In [14]:
tot_email = 0
tot_name= 0
tot_no_match = 0
test_email = test.loc[:, "email"]
for dic_email in test_email:
    for email, name in dic_email.items():
        if name == "no_match":
            tot_no_match += 1
        else:
            tot_name += 1
        tot_email += 1
print(f"total email: {tot_email}\ntotal name: {tot_name}\ntotal no match: {tot_no_match}")

total email: 126777
total name: 122018
total no match: 4759


In [17]:
df_inter

,PT,AU,AF,TI,SO,DT,DE,ID,AB,C1,...,DI,SC,UT,CN,lst_DT,lst_Target,lst_SDG,lst_AF,lst_EM,email
6,J,"Kao, CH; Wang, CC; Wang, HY","Kao, Chih-Han; Wang, Chien-Chih; Wang, Her -Yung",A neural-based predictive model of the compres...,COMPUTERS AND CONCRETE,Article,waste LCD concrete; compressive strength; back...,MECHANICAL-PROPERTIES; REGRESSION,The Taiwanese liquid crystal display (LCD) ind...,"[Kao, Chih-Han] Natl Quemoy Univ, Dept Civil E...",...,10.12989/cac.2017.19.5.457,Computer Science; Construction & Building Tech...,WOS:000403196900003,"Taiwan, Taiwan, Taiwan",{AI},"{12.5, 15.1}","{SDG15, SDG12}","[Wang, Chien-Chih, Wang, Her -Yung, Kao, Chih-...",[ccw@gcloud.csu.edu.tw],"{'ccw@gcloud.csu.edu.tw': 'Wang, Chien-Chih'}"
28,C,"Nugra, H; Abad, A; Fuertes, W; Galarraga, F; A...","Nugra, Hugo; Abad, Alejandra; Fuertes, Walter;...",A Low-cost IoT Application for the Urban Traff...,2016 IEEE/ACM 20TH INTERNATIONAL SYMPOSIUM ON ...,Proceedings Paper,Congestion of vehicular traffic; Wireless Sens...,INTERNET; SYSTEM; THINGS,Congestion of vehicular traffic in the cities ...,"[Nugra, Hugo; Abad, Alejandra; Fuertes, Walter...",...,10.1109/DS-RT.2016.24,Computer Science,WOS:000391589800021,"Ecuador, Ecuador, Ecuador, Ecuador, Ecuador, E...",{AI},{12.5},{SDG12},"[Toulkeridis, Theofilos, Galarraga, Fernando, ...","[henugra@espe.edu.ec, maabad2@espe.edu.ec, wmf...","{'henugra@espe.edu.ec': 'Nugra, Hugo', 'ttoulk..."
90,J,"Peng, YQ; Xiao, YX; Fu, ZT; Dong, YH; Zheng, Y...","Peng, Yaoqi; Xiao, Yingxin; Fu, Zetian; Dong, ...",Precision irrigation perspectives on the susta...,JOURNAL OF CLEANER PRODUCTION,Article,Field crop; Water demand prediction; Irrigatio...,REGRESSION-MODEL; DRIP IRRIGATION; NEURAL-NETW...,There is a global shortage of fresh water to m...,"[Peng, Yaoqi; Xiao, Yingxin; Fu, Zetian; Dong,...",...,10.1016/j.jclepro.2019.04.347,Science & Technology - Other Topics; Engineeri...,WOS:000474676300032,"China, China, China, China, China, China, Chin...",{AI},"{12.5, 7.3, 12.1, 2.4}","{SDG7, SDG12, SDG2}","[Zheng, Yongjun, Xiao, Yingxin, Dong, Yuhong, ...",[lxxcau@cau.edu.cn],"{'lxxcau@cau.edu.cn': 'Li, Xinxing'}"
101,J,"Alvarez-de-los-Mozos, E; Renteria-Bilbao, A; D...","Alvarez-de-los-Mozos, Esther; Renteria-Bilbao,...",WEEE Recycling and Circular Economy Assisted b...,APPLIED SCIENCES-BASEL,Article,human-robot collaboration; collaborative robot...,ELECTRONIC EQUIPMENT WEEE,Considering the amount of waste of electrical ...,"[Alvarez-de-los-Mozos, Esther; Diaz-Martin, Fe...",...,10.3390/app10144800,Chemistry; Engineering; Materials Science; Phy...,WOS:000554798600001,"Spain, Spain, Spain",{robotics},{12.5},{SDG12},"[Alvarez-de-los-Mozos, Esther, Renteria-Bilbao...","[esther.alvarez@deusto.es, arantxa.renteria@te...","{'fernando.diaz@deusto.es': 'Diaz-Martin, Fern..."
218,J,"Laskurain-Iturbe, I; Arana-Landin, G; Landeta-...","Laskurain-Iturbe, Iker; Arana-Landin, German; ...",Exploring the influence of industry 4.0 techno...,JOURNAL OF CLEANER PRODUCTION,Article,Industry 4; 0; Circular economy; Literature re...,SMART PRODUCTION; SUSTAINABILITY; CHALLENGES; ...,"In the last decade, both Industry 4.0 technolo...","[Laskurain-Iturbe, Iker; Arana-Landin, German]...",...,10.1016/j.jclepro.2021.128944,Science & Technology - Other Topics; Engineeri...,WOS:000704509900001,"Spain, Spain, Spain, Spain","{additive_manufacturing, AI, IOT, robotics, bi...","{12.5, 7.3}","{SDG7, SDG12}","[Uriarte-Gallastegi, Naiara, Laskurain-Iturbe,...","[iker.laskurain@ehu.eus, g.arana@ehu.eus, bena...","{'iker.laskurain@ehu.eus': 'Laskurain-Iturbe, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1244072,J,"Strengers, Y; Nicholls, L","Strengers, Yolande; Nicholls, Larissa",Aesthetic pleasures and gendered tech-work in ...,MEDIA INTERNATIONAL AUSTRALIA,Article,aesthetics; digital housekeeping; gender; hous...,NaN,This article analyses visions of everyday life...,"[Strengers, Yolande; Nicholls, Larissa] RMIT U...",...,1

In [25]:
%%time
import tqdm
from collections import Counter
idx_sdg = {name:ind for ind, name in enumerate(df_inter.columns)}
dic_authors = dict()

def ini_dic_authors(e):
    dic_authors[e] = {"name":set(), "total_publications":0, "lst_publications":set(), "PY<2019":0, "PY>=2019":0, "total_citations":0,"keywords":Counter()}
    dic_authors[e].update({f"SDG{i}":0 for i in range(0,18)})
    dic_authors[e].update({''})
for row in tqdm.tqdm(df_inter.itertuples(index=False, name=None)):
    dic_emails = row[-1]

    for email, name in dic_emails.items():
        if email not in dic_authors:
            ini_dic_authors(email)
        # name
        dic_authors[email]['name'].add(name)
        # Nb publications
        dic_authors[email]['total_publications'] += 1
        # List of publications ==> UT
        ut = row[idx_sdg['UT']]
        dic_authors[email]['lst_publications'].add(ut)
        # Years
        py = row[idx_sdg['PY']]
        dic_authors[email]["years"].append(py)
        # total citations
        dic_authors[email]['total_citations'] += int(row[idx_sdg['TC']])
        # SDG
        set_sdg = row[idx_sdg['lst_SDG']]
        dic_authors[email]['SDG'].update(set_sdg)
        # DT
        set_dt = row[idx_sdg['lst_DT']]
        dic_authors[email]['DT'].update(set_dt)
        # WC <=> keywords
        keywords = row[idx_sdg['WC']]
        if type(keywords) == str:
            keywords = keywords.split("; ")
            dic_authors[email]['keywords'].update(keywords)


53494it [00:01, 48648.48it/s]

CPU times: user 1.07 s, sys: 27.7 ms, total: 1.1 s
Wall time: 1.1 s


In [23]:
dic_authors

{'ccw@gcloud.csu.edu.tw': {'name': {'Wang, Chien-Chih'},
  'total_publications': 1,
  'lst_publications': {'WOS:000403196900003'},
  'years': [2017],
  'total_citations': 4,
  'SDG': Counter({'SDG15': 1, 'SDG12': 1}),
  'DT': Counter({'AI': 1}),
  'keywords': Counter({'AI': 1})},
 'henugra@espe.edu.ec': {'name': {'Nugra, Hugo'},
  'total_publications': 1,
  'lst_publications': {'WOS:000391589800021'},
  'years': [2016],
  'total_citations': 6,
  'SDG': Counter({'SDG12': 1}),
  'DT': Counter({'AI': 1}),
  'keywords': Counter({'AI': 1})},
 'ttoulkeridis@espe.edu.ec': {'name': {'Toulkeridis, Theofilos',
   'Toulkkeridis, Theofilos'},
  'total_publications': 5,
  'lst_publications': {'WOS:000391589800021',
   'WOS:000454766500024',
   'WOS:000464415600017',
   'WOS:000469101300017',
   'WOS:000502762900010'},
  'years': [2016, 2018, 2017, 2020, 2018],
  'total_citations': 25,
  'SDG': Counter({'SDG12': 2, 'SDG15': 1, 'SDG13': 1, 'SDG17': 1, 'SDG11': 2}),
  'DT': Counter({'AI': 2,
         

In [26]:
df_authors = pd.DataFrame.from_dict(dic_authors, orient='index')
df_authors

,name,total_publications,lst_publications,years,total_citations,SDG,DT,keywords
ccw@gcloud.csu.edu.tw,"{Wang, Chien-Chih}",1,{WOS:000403196900003},[2017],4,"{'SDG15': 1, 'SDG12': 1}",{'AI': 1},"{'Computer Science, Interdisciplinary Applicat..."
henugra@espe.edu.ec,"{Nugra, Hugo}",1,{WOS:000391589800021},[2016],6,{'SDG12': 1},{'AI': 1},"{'Computer Science, Hardware & Architecture': ..."
ttoulkeridis@espe.edu.ec,"{Toulkeridis, Theofilos, Toulkkeridis, Theofilos}",5,"{WOS:000469101300017, WOS:000464415600017, WOS...","[2016, 2018, 2017, 2020, 2018]",25,"{'SDG12': 2, 'SDG15': 1, 'SDG13': 1, 'SDG17': ...","{'AI': 2, 'robotics': 1, 'computing_infrastruc...","{'Computer Science, Hardware & Architecture': ..."
jfgalarraga@espe.edu.ec,"{Galarraga, Fernando}",1,{WOS:000391589800021},[2016],6,{'SDG12': 1},{'AI': 1},"{'Computer Science, Hardware & Architecture': ..."
wmfuertes@espe.edu.ec,"{Fuertes, Walter}",5,"{WOS:000703889300025, WOS:000464415600017, WOS...","[2016, 2020, 2017, 2020, 2018]",21,"{'SDG12': 1, 'SDG15': 1, 'SDG17': 1, 'SDG11': 2}","{'AI': 2, 'robotics': 1, 'computing_infrastruc...","{'Computer Science, Hardware & Architecture': ..."
...,...,...,...,...,...,...,...,...
o.sullivan@ucl.ac.uk,"{Sullivan, Oriel}",1,{WOS:000485280200015},[2019],14,{'SDG5': 1},{'IOT': 1},{'Sociology': 1}
j.gershuny@ucl.ac.uk,"{Gershuny, Jonathan}",1,{WOS:000485280200015},[2019],14,{'SDG5': 1},{'IOT': 1},{'Sociology': 1}
cornwell@cornell.edu,"{Cornwell, Benjamin}",1,{WOS:000485280200015},[2019],14,{'SDG5': 1},{'IOT': 1},{'Sociology': 1}
jasmine.abdulcadir@hcuge.ch,"{Abdulcadir, Jasmine}",1,{WOS:000565956100003},[2020],6,{'SDG5': 1},{'additive_manufacturing': 1},{'Urology & Nephrology': 1}


In [30]:
df_authors[df_authors['total_publications'] > 1]

,name,total_publications,lst_publications,years,total_citations,SDG,DT,keywords
ttoulkeridis@espe.edu.ec,"{Toulkeridis, Theofilos, Toulkkeridis, Theofilos}",5,"{WOS:000469101300017, WOS:000464415600017, WOS...","[2016, 2018, 2017, 2020, 2018]",25,"{'SDG12': 2, 'SDG15': 1, 'SDG13': 1, 'SDG17': ...","{'AI': 2, 'robotics': 1, 'computing_infrastruc...","{'Computer Science, Hardware & Architecture': ..."
wmfuertes@espe.edu.ec,"{Fuertes, Walter}",5,"{WOS:000703889300025, WOS:000464415600017, WOS...","[2016, 2020, 2017, 2020, 2018]",21,"{'SDG12': 1, 'SDG15': 1, 'SDG17': 1, 'SDG11': 2}","{'AI': 2, 'robotics': 1, 'computing_infrastruc...","{'Computer Science, Hardware & Architecture': ..."
cjvillacis@espe.edu.ec,"{Villacis, Cesar}",3,"{WOS:000454766500024, WOS:000464415600017, WOS...","[2016, 2017, 2018]",16,"{'SDG12': 1, 'SDG17': 1, 'SDG11': 1}","{'AI': 1, 'computing_infrastructure': 1, 'IOT'...","{'Computer Science, Hardware & Architecture': ..."
hmaules@espe.edu.ec,"{Aules, Hernan}",2,"{WOS:000464415600017, WOS:000391589800021}","[2016, 2017]",11,"{'SDG12': 1, 'SDG17': 1}","{'AI': 1, 'computing_infrastructure': 1}","{'Computer Science, Hardware & Architecture': ..."
lxxcau@cau.edu.cn,"{Li, Xinxing}",2,"{WOS:000504632600003, WOS:000474676300032}","[2019, 2020]",39,"{'SDG7': 1, 'SDG12': 1, 'SDG2': 2}",{'AI': 2},{'Green & Sustainable Science & Technology': 2...
...,...,...,...,...,...,...,...,...
harika.a@research.iiit.ac.in,"{Abburi, Harika}",2,"{WOS:000685817800001, WOS:000728362800036}","[2020, 2021]",1,{'SDG5': 2},{'AI': 2},"{'Computer Science, Artificial Intelligence': ..."
ttamati@indiana.edu,"{Tamati, Terrin N.}",2,"{WOS:000344907000009, WOS:000324566900010}","[2013, 2014]",68,{'SDG5': 2},{'AI': 2},"{'Audiology & Speech-Language Pathology': 2, '..."
ghosal.arijit@yahoo.com,"{Ghosal, Arijit}",3,"{WOS:000589273800009, WOS:000443206600096, WOS...","[2014, 2017, 2019]",15,{'SDG5': 3},{'AI': 3},"{'Computer Science, Artificial Intelligence': ..."
suchibrota@gmail.com,"{Dutta, Suchibrota}",2,"{WOS:000443206600096, WOS:000350814200130}","[2014, 2017]",14,{'SDG5': 2},{'AI': 2},"{'Computer Science, Artificial Intelligence': ..."


In [1]:
df_inter

NameError: name 'df_inter' is not defined

In [1]:
import pandas as pd
df_all_path = "../data/dataframes/all_nodups.pkl"
df_all = pd.read_pickle(df_all_path)


In [4]:
df_all

,PT,AU,AF,TI,SO,DT,DE,ID,AB,C1,...,Z9,PY,WC,DI,SC,UT,CN,lst_DT,lst_Target,lst_SDG
0,J,"Gusiatin, ZM; Kulikowska, D; Klik, B","Gusiatin, Zygmunt M.; Kulikowska, Dorota; Klik...",New-Generation Washing Agents in Remediation o...,INTERNATIONAL JOURNAL OF ENVIRONMENTAL RESEARC...,Review,soil washing; soil flushing; heavy metals; soi...,DISSOLVED ORGANIC-MATTER; REMOVE HEAVY-METALS;...,Soil quality is seriously reduced due to chemi...,"[Gusiatin, Zygmunt M.; Kulikowska, Dorota; Kli...",...,16.0,2020,"Environmental Sciences; Public, Environmental ...",10.3390/ijerph17176220,"Environmental Sciences & Ecology; Public, Envi...",WOS:000570328900001,"Poland, Poland, Poland",{},{12.5},{SDG12}
1,J,"Vitko, TG; Suffet, IH","Vitko, Tadeo G.; Suffet, Irwin H. (Mel)",Corrective factors applied to reduced sulfur c...,WATER ENVIRONMENT RESEARCH,Article,odor treatment design; odors; reduced sulfur o...,NaN,To determine accurately odorant concentrations...,"[Vitko, Tadeo G.] 26362 Via Conchita, Mission ...",...,2.0,2021,"Engineering, Environmental; Environmental Scie...",10.1002/wer.1558,Engineering; Environmental Sciences & Ecology;...,WOS:000635505300001,"United States, United States",{},{12.5},{SDG12}
2,C,"Ntasiou, M; Andreou, E","Ntasiou, M.; Andreou, E.",The Standard of Industrial Symbiosis. Environm...,SUSTAINABLE SYNERGIES FROM BUILDINGS TO THE UR...,Proceedings Paper,Industrial; Business; Parks; Symbiosis; Standard,NaN,The siting of an Industrial park with environm...,"[Ntasiou, M.; Andreou, E.] Hellen Open Univ, S...",...,6.0,2017,Construction & Building Technology; Engineerin...,10.1016/j.proenv.2017.03.157,Construction & Building Technology; Engineerin...,WOS:000410925400094,"Greece, Greece",{},{12.5},{SDG12}
3,J,"Wu, LP; Zhang, SR; Chen, MM; Liu, J; Ding, XD","Wu, Lipeng; Zhang, Shirong; Chen, Mengmeng; Li...",A sustainable option: Biochar addition can imp...,ENVIRONMENTAL TECHNOLOGY & INNOVATION,Article,Biochar; Phosphorus retention; Microbial bioma...,AGRICULTURAL SOILS; EXTRACTION METHOD; AVAILAB...,Application of straw and its derived biochar h...,"[Wu, Lipeng; Zhang, Shirong; Chen, Mengmeng; D...",...,1.0,2021,Biotechnology & Applied Microbiology; Engineer...,10.1016/j.eti.2021.102070,Biotechnology & Applied Microbiology; Engineer...,WOS:000744257400019,"China, China, China, China, Canada",{},{12.5},{SDG12}
4,C,"Staicu, D","Staicu, Daniela",Contributions of social entrepreneurship to te...,PROCEEDINGS OF THE INTERNATIONAL CONFERENCE ON...,Proceedings Paper,textile waste prevention; clothing reuse; soci...,ENVIRONMENTAL-IMPACT; FASHION; REUSE,"The current system for producing, distributing...","[Staicu, Daniela] Bucharest Univ Econ Studies,...",...,1.0,2019,Business,10.2478/picbe-2019-0009,Business & Economics,WOS:000501603000009,Romania,{},"{12.5, 13.0}","{SDG13, SDG12}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2798083,C,"Dieffenderfer, J; Wilkins, M; Hood, C; Beppler...","Dieffenderfer, James; Wilkins, Michael; Hood, ...",Towards a Sweat-based Wireless and Wearable El...,2016 IEEE SENSORS,Proceedings Paper,potentiostat; electrochemical; wearable sensin...,LACTATE CONCENTRATIONS; BIOSENSORS; BLOOD,The analysis of biomarkers within sweat is adv...,"[Dieffenderfer, James; Wilkins, Michael; Hood,...",...,9.0,2016,"Engineering, Electrical & Electronic; Remote S...",NaN,Engineering; Remote Sensing,WOS:000399395700065,"United States, United States, United States, U...",{IOT},{},{}
2798084,C,"Dinesh, K; Xiong, ML; Adams, J; Dorsey, R; Sha...","Dinesh, Karthik; Xiong, Mulin; Adams, Jamie; D...",SIGNAL ANALYSIS FOR DETECTING MOTOR SYMPTOMS I...,2016 IEEE WESTERN NEW YORK IMAGE AND SIGNAL PR...,Proceedings Paper,Wearable sensors; Parkinson's; Huntington's; a...,NaN,We report on a pilot study for signal processi...,"[Dinesh, Karthik; Sharma, Gaurav] Univ Rochest...",...,1.0,2016,"Engineering, Electrical & Electronic",NaN,Engineering,WOS:000402630000005,"United States, United States, United States

In [2]:
test = df_all.sample(5)

In [3]:
test.to_dict(orient="index")

{1430581: {'PT': 'J',
  'AU': 'Zhang, YL; Li, JC; Zhou, H; Liu, YQ; Han, DD; Sun, HB',
  'AF': 'Zhang, Yong-Lai; Li, Ji-Chao; Zhou, Hao; Liu, Yu-Qing; Han, Dong-Dong; Sun, Hong-Bo',
  'TI': 'Electro-responsive actuators based on graphene',
  'SO': 'INNOVATION',
  'DT': 'Review',
  'DE': 'Electro-responsive actuators; Graphene; Electrostatic actuation; Electrothermal actuation; Ionic actuation',
  'ID': 'IONIC POLYMER ACTUATOR; BIMORPH ACTUATOR; CARBON NANOTUBE; LASER FABRICATION; SOFT ACTUATORS; OXIDE; HYDROGELS; FILM; STRENGTH; SURFACE',
  'AB': 'Electro-responsive actuators (ERAs) hold great promise for cutting-edge applications in e-skins, soft robots, unmanned flight, and in vivo surgery devices due to the advantages of fast response, precise control, programmable deformation, and the ease of integration with control circuits. Recently, considering the excellent physical/chemical/mechanical properties (e.g., high carrier mobility, strong mechanical strength, outstanding thermal con

'Gamboa, Emily; Broadbent, Eliza; Quintana, Noyra; Callaway, Sarah; Donoso, Paola; Linehan, Mary; Wibowo, Lindawati; Santika, Otte; West, Josh H.; Hall, P. Cougar; Crookston, Benjamin T.'

In [2]:
df = df_all[['AF', 'EM', 'PY', 'UT', 'WC', 'TC', 'lst_SDG', 'lst_DT']]
df

,AF,EM,PY,UT,WC,TC,lst_SDG,lst_DT
0,"Gusiatin, Zygmunt M.; Kulikowska, Dorota; Klik...",mariusz.gusiatin@uwm.edu.pl; dorotak@uwm.edu.p...,2020,WOS:000570328900001,"Environmental Sciences; Public, Environmental ...",14,{SDG12},{}
1,"Vitko, Tadeo G.; Suffet, Irwin H. (Mel)",tvitko@cox.net,2021,WOS:000635505300001,"Engineering, Environmental; Environmental Scie...",2,{SDG12},{}
2,"Ntasiou, M.; Andreou, E.",eandreou3@gmail.com,2017,WOS:000410925400094,Construction & Building Technology; Engineerin...,6,{SDG12},{}
3,"Wu, Lipeng; Zhang, Shirong; Chen, Mengmeng; Li...",xiaodongding2004@163.com,2021,WOS:000744257400019,Biotechnology & Applied Microbiology; Engineer...,1,{SDG12},{}
4,"Staicu, Daniela",Danielastaicu@gmail.com,2019,WOS:000501603000009,Business,1,"{SDG12, SDG13}",{}
...,...,...,...,...,...,...,...,...
2798083,"Dieffenderfer, James; Wilkins, Michael; Hood, ...",mdaniel6@ncsu.edu; aybozkur@ncsu.edu,2016,WOS:000399395700065,"Engineering, Electrical & Electronic; Remote S...",9,{},{IOT}
2798084,"Dinesh, Karthik; Xiong, Mulin; Adams, Jamie; D...",NaN,2016,WOS:000402630000005,"Engineering, Electrical & Electronic",1,{},{IOT}
2798085,"Ding, Caiwen; Heidari, Soroush; Wang, Yanzhi; ...",cading@syr.edu; soroush.heidari@okstate.edu; y...,2016,WOS:000390094700044,"Engineering, Electrical & Electronic",9,{},{IOT}
2798086,"Ding, Caiwen; Li, Hongjia; Hu, Jingtong; Liu, ...",cading@syr.edu; hli42@syr.edu; jthu@okstate.ed...,2016,WOS:000391829200039,"Computer Science, Hardware & Architecture; Eng...",1,{},{IOT}


In [4]:
df_dict = df.to_dict('records')


In [5]:
df_dict

[{'AF': 'Gusiatin, Zygmunt M.; Kulikowska, Dorota; Klik, Barbara',
  'EM': 'mariusz.gusiatin@uwm.edu.pl; dorotak@uwm.edu.pl; barbara.klik@uwm.edu.pl',
  'PY': 2020,
  'UT': 'WOS:000570328900001',
  'WC': 'Environmental Sciences; Public, Environmental & Occupational Health',
  'TC': 14,
  'lst_SDG': {'SDG12'},
  'lst_DT': set()},
 {'AF': 'Vitko, Tadeo G.; Suffet, Irwin H. (Mel)',
  'EM': 'tvitko@cox.net',
  'PY': 2021,
  'UT': 'WOS:000635505300001',
  'WC': 'Engineering, Environmental; Environmental Sciences; Limnology; Water Resources',
  'TC': 2,
  'lst_SDG': {'SDG12'},
  'lst_DT': set()},
 {'AF': 'Ntasiou, M.; Andreou, E.',
  'EM': 'eandreou3@gmail.com',
  'PY': 2017,
  'UT': 'WOS:000410925400094',
  'WC': 'Construction & Building Technology; Engineering, Civil; Environmental Sciences',
  'TC': 6,
  'lst_SDG': {'SDG12'},
  'lst_DT': set()},
 {'AF': 'Wu, Lipeng; Zhang, Shirong; Chen, Mengmeng; Liu, Jian; Ding, Xiaodong',
  'EM': 'xiaodongding2004@163.com',
  'PY': 2021,
  'UT': 'WOS:0

In [3]:
df.to_pickle("../data/dataframes/all_nodups_shorten.pkl")

In [6]:
import pickle
with open("../data/dataframes/all_nodups_shorten_dic.pkl", "wb") as f:
    pickle.dump(df_dict, f)

In [2]:
import pickle
with open('../data/dataframes/all_nodups_shorten_dic.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

In [8]:
size = len(loaded_dict)

n = 1
n_start = 0
n_end = 100000
dic_authors = None
while n_end <= size:
    print(n_end)
    smaller_dic = loaded_dict[n_start: n_end]

    with open(f"../data/dataframes/temp/slice{n}.pkl", "wb") as f:
        pickle.dump(smaller_dic, f)
    n_start += 100000
    n += 1
    if n_end + 100000 > size and n_end != size:
        n_end = size
    else:
        n_end += 100000


100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2798088


In [1]:
import copy

s = {4,5,7,8}
for elem in copy.copy(s):
    s.remove(elem)


In [3]:
s = {4,5,7,8}
for elem in (s):
    s.remove(elem)

RuntimeError: Set changed size during iteration

In [6]:
a = "slice25.pkl"
b = "slice1.pkl"
ind = b.split(".")[0][5:7]

In [7]:
ind

'1'